In [1]:
from textgrad import (
    set_backward_engine,
    BlackboxLLM,
    Variable,
)

In [2]:
from dotenv import load_dotenv
load_dotenv("/media/uberdev/ddrv/gitFolders/python_de_learners_data/.env")

True

In [3]:
set_backward_engine(engine="gpt-4o-mini", override=True)

In [36]:
model = BlackboxLLM(engine="gpt-4o-mini")

question = ("If it takes 1hr to dry 25 shirts under the sun,"
            "How long will it take to dry 30 shirts under sun? reason step by step")
# the parenthesis is the way to break the sentences, 
# not a tuple
diamond_question = ("I put a diamond in a cup and then placed the cup upside down on my bed,"
                    "Later I came back, took the cup, and put it in the fridge. Where is the diamond?")

question_var = Variable(value=diamond_question,
                        role_description="Question to LLM",
                        requires_grad=False)

In [34]:
diamond_question

'I put a diamond in a cup and then placed the cup upside down on my bed,Later I came back, took the cup, and put it in the fridge. Where is the diamond?Make correct assumptions'

In [37]:
answer = model(question_var)
answer

Variable(value=The diamond is still in the cup. When you placed the cup upside down on your bed, the diamond would have remained inside the cup. When you later took the cup and put it in the fridge, the diamond went with it. So, the diamond is now in the cup, which is in the fridge., role=response from the language model, grads=set())

In [38]:
from textgrad import TGD

answer.set_role_description("concise and accurate answer to the question")

# defining the loss function, Text Grad Descent
optimizer = TGD(parameters=[answer],engine='gpt-4o')
valuation_instrn = (f"Here's a question: {diamond_question}."
                    "Evaluate any given answer to ths question"
                    "Be smart, logical and very critical"
                    "Just provide concise feedback")

In [39]:
from textgrad import TextLoss

loss_fn = TextLoss(eval_system_prompt=valuation_instrn)

In [40]:
loss = loss_fn(answer)
loss

Variable(value=The answer is correct. The diamond remained inside the cup throughout the described actions. When the cup was placed upside down, the diamond did not fall out, and when the cup was moved to the fridge, the diamond was taken along with it. Thus, the diamond is indeed in the cup, which is now in the fridge. The reasoning is logical and follows the sequence of events accurately., role=response from the language model, grads=set())

In [41]:
loss.backward()

In [42]:
optimizer.step()

In [43]:
answer

Variable(value=The diamond is in the fridge. When you placed the cup upside down on your bed, the diamond remained inside. Later, when you moved the cup to the fridge, the diamond went with it., role=concise and accurate answer to the question, grads={Variable(value=To improve the concise and accurate answer, consider the following feedback:

1. **Conciseness**: The response could be more succinct. While the explanation is accurate, it could be streamlined by removing redundant phrases. For example, the phrase "the diamond is still in the cup" could be shortened to "the diamond remains in the cup," which conveys the same meaning with fewer words.

2. **Clarity**: While the reasoning is logical, the structure could be enhanced for clarity. Presenting the sequence of events in a more straightforward manner could help. For instance, stating the final location of the diamond first ("The diamond is in the fridge") followed by a brief explanation of how it got there could improve the flow.



Diving into the primitives of TextGrad

In [44]:
# Variable is the main data structure
x = Variable("A sentnke with typo",
             role_description="The input sentence",
             requires_grad=True)

In [45]:
# engine is the LLM
from textgrad import get_engine
engine = get_engine("gpt-4o-mini")

In [46]:
engine.generate("How are you?")

"I'm just a computer program, so I don't have feelings, but I'm here and ready to help you! How can I assist you today?"

In [47]:
sys_prompt = Variable("Evaluate the sentence correctness",
                      role_description="System Prompt")

In [48]:
loss = TextLoss(sys_prompt, engine=engine,)

# How the loss is calculate with respect to a string?

In [50]:
l = loss(x)
# when sending through the loss, the sentence is corrected

In [52]:
l.backward()

In [53]:
from textgrad import TextualGradientDescent

optimizer = TextualGradientDescent(parameters=[x],
                                   engine=engine)

In [54]:
optimizer.step()

In [55]:
x.value

'This sentence contains a typo.'

In [56]:
optimizer.zero_grad()